# Run analysis of validation data

In [ ]:
%load_ext rpy2.ipython

### Import packages

In [ ]:
%%R
library(reshape2)
library(cluster) 
library(factoextra)
library(NbClust)
library(rjson)

library(gridExtra)
library(grid)
library(ggplot2)
library(ggfortify)
library(ggdendro)
library(dendextend)
library(mclust)
library(RColorBrewer)

library(magrittr)

### Lets set the experiment WD and simulation WD

In [ ]:
%%R
exp_wd <- "./Experiment_out/dense/"
sim_wd <- "./Simulation_result/dense/Experiment_Med_Cov_0/"
cbbPalette <- c("#EEEEEE", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#A6761D", "#666666")
cbbPalette_reduced <- cbbPalette[2:length(cbbPalette)]

### Inspect a Peak

#### Define Plotting Functions

In [ ]:
%%R
plot_pattern <- function(data, col_palette) {
    acum <- matrix(0, nrow=10,ncol=10)
    colour_idx <- 1
    colour_vec <- matrix(rep(0,10))
    for (pattern in data){
        p <-matrix(pattern)
        acum <- acum + (p %*% t(p))*colour_idx
        colour_vec <- colour_vec+p*colour_idx
        colour_idx<-colour_idx+1
    }
    plt<-c()
    plt$mat <- (ggplot(data = melt(acum), aes((11-Var1), Var2, fill=factor(value)))+
    geom_tile()+
    scale_x_continuous("modification", breaks=c(0:10)+0.5)+
    scale_y_continuous("modification", breaks=c(0:10)+0.5)+
    scale_fill_manual(values=col_palette, labels = c("None", c(1:10)))+
    guides(fill=guide_legend(title="Association",title.position = "left"))+
    coord_fixed() +
    theme(
        panel.grid.major = element_line(colour = "black"),
        legend.title = element_text(angle=90,hjust=0.5),
        legend.key.height = grid::unit(2.5,"cm"),
        legend.key.width = grid::unit(1.75,"cm"),
        legend.margin = grid::unit(0.25,"cm"),
        legend.box = "vertical",
        legend.box.margin = margin(c(10,10,10,10)),
        panel.background = element_rect(fill = NA),
        panel.ontop = TRUE,
        axis.text.x=element_blank(),
        axis.text.y=element_blank(),
        axis.ticks=element_line(size=0.4),
        panel.border=element_blank())
    )
    plt$colours <- colour_vec
    return(plt)
}

g_legend<-function(a.gplot){
    tmp <- ggplot_gtable(ggplot_build(a.gplot))
    leg <- which(sapply(tmp$grobs, function(x) x$name) == "guide-box")
    legend <- tmp$grobs[[leg]]                        
    return(legend)
}
                        
plot_association <- function(patt_no, patterns, palette) {
    
    plts <- list()
    
    patt_name <- paste0("pattern_",patt_no)
    peak_data <-data.frame(read.csv(paste0(sim_wd,"/",patt_name,"/","/peak_0/data.csv")))
    layout_mat <- matrix(0, nrow=10,ncol=3)
    layout_mat[,1] <- rep(1,10)
    layout_mat[,2] <- rep(2,10)
    layout_mat[,3] <- c(3:12)

    patt_plt <- plot_pattern(patterns[[patt_no+1]], cbbPalette)
    colours <- patt_plt$colours
    
    mat_plt <-patt_plt$mat
    
    legend <- g_legend(mat_plt)
    plts[[1]] <- legend
    plts[[2]] <- mat_plt+theme(legend.position="none")
    
    for (i in 0:9) {
        p <- (ggplot(subset(peak_data,modification==i), aes(x=position))+
                geom_histogram(colour="white", fill=cbbPalette[colours[i+1]+1],bins=150)+
                xlim(c(0,2100)) +
                ylim(c(0,20)))
        plts[[i+3]] <- p
    }
    pdf(paste0("association_pattern_",patt_no,".pdf"), width=31,height=15)
        grid.arrange(grobs=plts,
                     layout_matrix=layout_mat, widths=c(2,15,14))
        grid.rect(gp=gpar(fill=NA))
    dev.off()
} 

#### Plot Patterns for Peak

In [ ]:
%%R
patterns <- fromJSON(file=paste0(sim_wd, "/","patterns.json"))
for(i in c(0:7)) {
    plot_association(i, patterns, cbbPalette)
}

### Auxilliary Functions

In [ ]:
%%R
make_mat <- function(x) {
    b <- matrix(0, 10, 10)
    b[lower.tri(b, diag=FALSE)] <- x
    b <- t(b)
    b <- (b+t(b))
    return(b)
}

###jaccard
dist_jacc <- function(df){
    dd <- as.matrix(df)
    o_prod <- outer(split(dd, row(dd)), split(dd, row(dd)), FUN=Vectorize(jacc))
    return(o_prod)
}

jacc <- function(vec1,vec2) {
    m <- sum(abs(vec1-vec2))
    A <- make_mat(vec1) + make_mat(vec2)
    n <- nuclear_norm(A)
    return (m/n)
}

nuclear_norm <- function(A) {
    return(sum(diag(sqrt(A %*% t(A)))))
}

###hamming
dist_hamm <- function(df){
    dist_mat <- dist(df, method="euclidean")
    return(dist_mat)
}

### Plot Hierarchical Clustering

In [ ]:
%%R
clus_plot <- function(hc, g_truth, name, palette) {
    
    col_vec <- as.integer(factor(g_truth))
    
    dend <- as.dendrogram(hc) %>% 
            set("branches_k_color", k=8) %>%
            set("branches_lwd", 0.7)
    
    col_vec <- col_vec[as.integer(labels(dend))]    
    ggd1 <- as.ggdend(dend)
    
    clusters <- as.vector(cutree(hc, 8))
    performance <- adjustedRandIndex(clusters, g_truth)
    
    y_scale <- max(ggd1$segments$y)/10
                      
    plt <- ggplot() +
            geom_segment(data = ggd1$segments, aes(x = x, y = y/y_scale, xend = xend, yend = yend/y_scale, color=factor(col))) +
            geom_tile(data = ggd1$labels, aes(x=x, y=(y-1), fill=factor(col_vec))) +
            scale_colour_brewer(palette = "Paired", name="Clustering Assignment", na.value="black") +
            scale_fill_brewer(palette="RdYlGn", name="Ground Truth") +
            scale_y_reverse(expand = c(0, 0)) +
            coord_polar(theta="x")  +
            ggtitle(paste0("Hierarchical Clustering: ", name)) +
            annotate("text",x=0,y=-2.5, label=paste0("ARI: ", performance), size=6)+
            theme(aspect.ratio = 1,
                    axis.line.y=element_blank(),
                    axis.ticks.y=element_blank(),
                    axis.text.y=element_blank(),
                    axis.title.y=element_blank(),
                    axis.line.x=element_blank(),
                    axis.ticks.x=element_blank(),
                    axis.text.x=element_blank(),
                    axis.title.x=element_blank(),
                    panel.grid  = element_blank(),
                    panel.border = element_blank(),
                    plot.title = element_text(size = 8, hjust = 0.5, face = "bold"),
                    legend.position="none")
    return(plt)
}

In [ ]:
%%R 
replicas <- list.dirs(path = exp_wd, full.names = TRUE, recursive = FALSE)
n_replicas <- length(replicas)
n_plots <- 4

data_grid <-list()
name_grid <-list()
replica_names <-list()

index <- 1;

for (replica in replicas){
    
    exp_df <- data.frame()
    patterns <- list.dirs(path = replica, full.names = TRUE, recursive = FALSE)
    
    for (pattern in patterns){
        pattern_name <- basename(pattern)
        dists <- data.frame(read.csv(file=paste0(pattern, '/', 'MMD_dists.csv'), header=TRUE, sep=","))
        dists$pattern <- pattern_name
        exp_df <- rbind(exp_df, dists)
    }
    
    dat_df <- exp_df[,2:46]
    
    ### Compute Jaccard distance
    jacc_d <- as.dist(dist_jacc(dat_df))
    ### Compute Hamming distance
    hamm_d <- as.dist(dist_hamm(dat_df))
    
    replica_list <- list(hclust(jacc_d, method = "ward.D2"),
                         hclust(jacc_d, method = "complete"),
                         hclust(hamm_d, method = "ward.D2"),
                         hclust(hamm_d, method = "complete"))
    name_list <- list("Jaccard/Ward", "Jaccard/Complete", "Hamming/Ward", "Hamming/Complete")
    
    data_grid[[index]] <- replica_list
    name_grid[[index]] <- name_list
    replica_names[[index]] <- paste0("Replica: ", basename(replica))
    
    index <- index+1
}

In [ ]:
%%R
pl_grid <- list()
for (i in c(1:n_replicas)){
    
    offset <- (i-1)*(n_plots+1)
    
    pl_grid[[offset + 1]] <- textGrob(replica_names[[i]], rot=90)
    
    for (j in c(1:n_plots)){
        pl_grid[[offset + (1+j)]] <- clus_plot(data_grid[[i]][[j]],exp_df$pattern, name_grid[[i]][[j]], cbbPalette_reduced)
    }
}

pdf(paste0("Clustering_no_rescale.pdf"), width=n_plots*5+1,height=5*n_replicas)
grid.arrange(grobs=pl_grid,
            nrow=n_replicas,
            ncol=(n_plots+1),
            widths=c(1,rep(5,n_plots)))
grid.rect(gp=gpar(fill=NA))
dev.off()

### Plot MDS Projections

In [ ]:
%%R

mds_plot <- function(dists, lab, name, palette) { 
    ### Compute MDS
    cmds.df <- as.data.frame(cmdscale(dists))
    cmds.df$labels <- lab
    ### Plot MDS
    plt<- ggplot(cmds.df, aes(V1, V2)) +
        geom_point(shape=23, aes(fill = factor(labels)), color="white", size=3)+
        scale_fill_manual(values = palette, name = "groups") +
        ggtitle(paste0("MDS Projection of ",name)) +
        theme(aspect.ratio = 1,
              plot.title = element_text(size = 8, hjust = 0.5, face = "bold"))
    return(plt)
}

replicas <- list.dirs(path = exp_wd, full.names = TRUE, recursive = FALSE)
n_replicas <- length(replicas)

pl_grid <- list()
index <- 0

for (replica in replicas){
    
    pl <- list()
    
    exp_df <- data.frame()
    patterns <- list.dirs(path = replica, full.names = TRUE, recursive = FALSE)
    
    for (pattern in patterns){
        pattern_name <- basename(pattern)
        dists <- data.frame(read.csv(file=paste0(pattern, '/', 'MMD_dists.csv'), header=TRUE, sep=","))
        dists$pattern <- pattern_name
        exp_df <- rbind(exp_df, dists)
    }
    
    dat_df <- exp_df[,2:46]
    
    ### Compute Jaccard distance
    jacc_d <- dist_jacc(dat_df)
    ### Compute Hamming distance
    hamm_d <- dist_hamm(dat_df)
        
    pl[[1]] <- mds_plot(jacc_d, exp_df$pattern, "Jaccard Distance", palette = cbbPalette_reduced)
    pl[[2]] <- mds_plot(hamm_d, exp_df$pattern, "Hamming Distance", palette = cbbPalette_reduced)
    
    pl_grid[[index*4 + 1]] <- textGrob(paste0("Replica: ", basename(replica)), rot=90)
    pl_grid[[index*4 + 2]] <- g_legend(pl[[1]])
    pl_grid[[index*4 + 3]] <- pl[[1]]+theme(legend.position="none", aspect.ratio = 1, plot.title = element_text(size = 8, hjust = 0.5, face = "bold"))
    pl_grid[[index*4 + 4]] <- pl[[2]]+theme(legend.position="none", aspect.ratio = 1, plot.title = element_text(size = 8, hjust = 0.5, face = "bold"))
    
    index <- index+1
}

pdf(paste0("Projections_no_rescale.pdf"), width=2*5+2,height=5*n_replicas)
grid.arrange(grobs=pl_grid,
            nrow=n_replicas,
            ncol=4,
            widths=c(1,1,rep(5,2)))
grid.rect(gp=gpar(fill=NA))
dev.off()
print(pl_grid)